## 总领

1. 使用图(Graph)表示计算任务  
2. 会话（Session）的上下文中执行图
3. 使用tensor表示数据
4. 通过变量（Variable）维护状态
5. 使用feed和fetch为任意操作赋值或者从其中获取数据

tensorflow图描述计算过程，为了进行计算，图必须在会话中启动，图中的节点为op,会话将图的op分发到CPU或GPU等设备上，同时执行op.  
一个Op获得0个或多个Tensor，每个Tensor是多维数组，是numpy.ndarray.

### 构建图
创建源op，输出传入其他op做运算
例子：构建三个op,分别是两个constant() op与一个matmul()op

In [1]:
import tensorflow as tf
matrix1 = tf.constant([[3,3]])
matrix2 = tf.constant([[2],[2]])
product = tf.matmul(matrix1,matrix2)

### 在一个会话中启动图
首先创建Session对象，如果无任何创建参数，默认启动默认图（Default Graph）  
执行过程是自动化的，会话负责传递op所需要的全部输入，op通常是并发执行，会触发全过程

In [2]:
sess = tf.Session()
result = sess.run(product)
print(result)
sess.close()

[[12]]


In [3]:
# 等价方法：
with tf.Session() as sess:
    result = sess.run(product)
    print(result)

[[12]]


指定cpu还是gpu资源
* "/cpu:0":调用cpu
* "/gpu:0":调用第一个GPU
* "/gpu:1":调用第二个GPU

默认有GPU先用GPU，但是只默认使用一个GPU，如果有多个GPU，余下GPU需要自己指派给op

In [4]:
with tf.Session() as sess:
    with tf.device('/cpu:0'):
        matrix1 = tf.constant([[3., 3.]])
        matrix2 = tf.constant([[2.],[2.]])
        product = tf.matmul(matrix1, matrix2)
        result = sess.run(product)
        print(result)

[[12.]]


### 交互式使用，避免Sess单变量持有会话

In [5]:
sess = tf.InteractiveSession()
x = tf.Variable([1,2])
a = tf.constant([3,3])
x.initializer.run()  # 初始化x
sub = tf.subtract(x, a)
print(sub.eval())

[-2 -1]


## Tensor
一个tensor包括一个静态类型rank和一个shape  
rank是张量的阶数，0：纯量，1:向量，2：矩阵，.....   
shape是张量的形状，在每一个维度的长度？
数据类型

## 变量

In [8]:
# 创建一个变量，初始化为标量 0
state = tf.Variable(0, name = 'counter')  # 变量需要初始化，但是可以用assign改值
# 创建一个op,其作用是使state增加1
one = tf.constant(1)
new_values = tf.add(state, one)  # 这是个op
update = tf.assign(state, new_values)  # 这他妈也是op

# 启动图后，变量必须经过‘初始化' op初始化
# 首先必须增加一个‘初始化‘ op到图中
init_op = tf.global_variables_initializer()

# 启动图， 运行op
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(state))
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

0
1
2
3


## Fetch
取回变量，可以一次性取回多个 

In [17]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
    print(sess.run([mul, intermed]))

[21.0, 7.0]


## Feed
feed 使用一个 tensor 值临时替换一个操作的输出结果. 你可以提供 feed 数据作为 run() 调用的参数. feed 只在调用它的方法内有效, 方法结束, feed 就会消失. 最常见的用例是将某些特殊的操作指定为 "feed" 操作, 标记的方法是使用 tf.placeholder() 为这些操作创建占位符.

In [21]:
input1 = tf.placeholder(tf.float32)  # 就是个要填数的空变量
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print(sess.run([output], feed_dict = {input1:[7.], input2: [2.]}))

[array([14.], dtype=float32)]
